In [34]:
import pandas as pd

In [35]:
df=pd.read_csv('./data/cubic_zirconia.csv')
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z,price
0,1,0.30,Ideal,E,SI1,62.1,58.0,4.27,4.29,2.66,499
1,2,0.33,Premium,G,IF,60.8,58.0,4.42,4.46,2.70,984
2,3,0.90,Very Good,E,VVS2,62.2,60.0,6.04,6.12,3.78,6289
3,4,0.42,Ideal,F,VS1,61.6,56.0,4.82,4.80,2.96,1082
4,5,0.31,Ideal,F,VVS1,60.4,59.0,4.35,4.43,2.65,779


In [36]:
df=df.drop('Unnamed: 0', axis=1)

In [37]:
## dependent and independent features
X=df.drop(labels=['price'],axis=1)
Y=df[['price']]

In [38]:
Y

,price
0,499
1,984
2,6289
3,1082
4,779
...,...
26962,5408
26963,1114
26964,1656
26965,682


In [39]:
##columns ordinal-encoded and scaling
categorical_columns=X.select_dtypes(include='object').columns
numerical_columns=X.select_dtypes(exclude='object').columns

In [40]:
cut_categories=["Fair","Good","Very Good","Premium","Ideal"]
clarity_categories=["I1","SI2","SI1","VS2","VS1","VVS2","VVS1","IF"]
color_categories=["D","E","F","G","H","I","J"]

In [41]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [42]:
##numerical pipeline & categorical pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy="median")),
        ('scaler',StandardScaler())
    ]
)

cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy="most_frequent")),
        ('OrdinalEncoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_columns),
    ('cat_pipeline',cat_pipeline,categorical_columns)
])





In [43]:
## Train Test Split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42)


In [44]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.fit_transform(X_test),columns=preprocessor.get_feature_names_out())

In [45]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.787964,-0.966850,3.365840,0.964420,0.934381,0.813401,-1.717499,-1.523957,-1.247448
1,0.851153,-0.461986,-1.095845,1.008892,1.041947,0.957195,0.981578,-0.354480,1.791324
2,-1.023443,-0.101369,0.688829,-1.259163,-1.252793,-1.257232,-0.817807,1.399736,1.183569
3,2.936379,0.547743,-0.649677,2.298570,2.252064,2.366376,0.981578,1.399736,-1.247448
4,-1.044506,0.259249,-0.203508,-1.294740,-1.279684,-1.257232,0.981578,0.814997,-0.031940


In [46]:
## Model_training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [47]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [48]:
regression.coef_

array([[ 5134.94904902,   -67.41049312,   -63.44838633, -2608.5353019 ,
         1988.31634422,  -361.0516354 ,   138.46524479,  -572.15521544,
          817.73546177]])

In [49]:
regression.intercept_

array([3918.29561348])

In [50]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [51]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    y_pred=model.predict(X_test)
    mae,rmse,r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model Training Perfromane")
    print("RMSE :",rmse)
    print("MAE :",mae)
    print("R2_score :",r2_square*100)

    r2_list.append(r2_square)
    print("\n\n")


LinearRegression
Model Training Perfomane
RMSE 1532.9431789841678
MAE 832.4902831631402
R2_score 85.80305308098485





d:\ML_Project\Diamond_Price_Prediction\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.581e+08, tolerance: 3.029e+07
  model = cd_fast.enet_coordinate_descent(


Lasso
Model Training Perfomane
RMSE 1350.5289049534458
MAE 821.5334176672982
R2_score 88.98078615547858



Ridge
Model Training Perfomane
RMSE 1511.558786047896
MAE 831.1485374853019
R2_score 86.1963821358546



ElasticNet
Model Training Perfomane
RMSE 1751.009774506175
MAE 1087.399292682636
R2_score 81.47663072576424



